In [ ]:
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

base_dir = ".\\signlang\\signlang" #MUST HAVE SIGNLANG DIR in current directory as this file python file


test_directory = os.path.join(base_dir,"test")


train_datagen = ImageDataGenerator(rescale= 1.0/255)
test_datagen = ImageDataGenerator(rescale= 1.0/255)

train_directory = os.path.join(base_dir,"train")
train_generator = train_datagen.flow_from_directory(train_directory, target_size=(150,150), 
                                                    shuffle=True, batch_size=20, class_mode='categorical')


val_directory = os.path.join(base_dir,"val")
val_generator = test_datagen.flow_from_directory(val_directory, target_size=(150,150), batch_size=20, 
                                                 shuffle=True, class_mode='categorical')

test_directory = os.path.join(base_dir,"test")
test_generator = test_datagen.flow_from_directory(test_directory, target_size=(150,150), batch_size=20, 
                                                 shuffle=True, class_mode='categorical')

In [ ]:
#Displaying randomised batch of images within datasets

import matplotlib.pyplot as plt

for data_batch, label_batch in train_generator:
    break

    
fig=plt.figure(figsize=(20,20))
columns = 3
rows = 3
for i in range (1,10):
    img = data_batch[i]
    ax = fig.add_subplot(rows, columns, i)
    plt.subplots_adjust(hspace=None, wspace = 0.01)
    ax.title.set_text(str(i)+ "label=" +str(label_batch[i]))
    plt.imshow(img)
plt.show()

In [ ]:
#Model

from keras import models, layers
from tensorflow.keras import optimizers

network = models.Sequential()
network.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(150,150,3)))
network.add(layers.MaxPooling2D(2, 2))

network.add(layers.Flatten())
network.add(layers.Dense(128, activation="relu"))
network.add(layers.Dense(10, activation="softmax"))

network.compile(loss="categorical_crossentropy", optimizer=optimizers.RMSprop(lr=1e-4), metrics=["accuracy"])

network.summary()



In [ ]:
history = network.fit_generator(train_generator, steps_per_epoch=20, epochs=20, 
                             validation_data=val_generator, validation_steps=20)


In [ ]:
#SAVING AND LOADING history of fitting the model.
import pickle
dictionary_data = history.history
history_file = open("ASL_History_Data", "wb")
pickle.dump(dictionary_data, history_file)
history_file.close()


In [ ]:
import pickle
history_file = open("ASL_History_Data", "rb")
history = pickle.load(history_file)
history_file.close()

In [ ]:
#Displaying results onto metaplot graph
Training_history_info = [history["loss"], history["accuracy"]]
Validation_history_info = [history["val_loss"], history["val_accuracy"]]

epochs = range(1,len(Training_history_info[0])+1)

plt.plot(epochs,Training_history_info[0],'b', label='Train Loss')
plt.plot(epochs,Validation_history_info[0], 'c', label='Validation Loss')
plt.title('Loss')
plt.legend()
plt.figure()


plt.plot(epochs,Training_history_info[1],'b', label='Train Accuracy')
plt.plot(epochs,Validation_history_info[1], 'c', label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

plt.show()

In [ ]:
test_loss, test_accuracy = network.evaluate(test_generator, verbose=2)
print ("Overall accuracy: {0}".format(test_accuracy))